# Get_BS_Season.ipynb
The purpose of this notebook is to grab some useful parts of the Baseball Savant play-by-play data to merge with our Fangraphs season level data. This notebook produces a .csv file with season-level data for pitchers, and has some calculations run to get some helpful numbers for our later computations.

### Imports

In [3]:
import numpy as np
import pandas as pd
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

In [ ]:
#Load in data
data = pd.read_csv('./drive/MyDrive/RedsHackathon/savant_pitch_level.csv')

### Computing absolute delta win expectancy
We wanted to create some metric to measure how impactful the situation each pitcher is placed in is. To do this, we wanted to use win expectancy, but we also knew that if we aggregated it by pitcher, we would see a big difference between good pitchers placed in high-leverage situations and bad pitchers placed in those situations based on how effective the pitchers are. To do this, we took the absolute value of the win expectancy changes to separate out pitcher skill and focus on the situation.

In [ ]:
data['abs_delta_win_exp'] = abs(data['delta_home_win_exp'])

### Change ID Values
Next, we had to change the savant ID values to match our Fangraphs ID values, adding the year and whether or not the player is starting the game to our player ID.

In [ ]:
data['pitcher_ID'] = data['pitcher'].astype(str) + data['game_year'].astype(str) + data['sp_indicator'].astype(str)

### Change handedness to 0 and 1 values
We also wanted to work with handedness and find our which pitchers have a platoon advantage more often than others. To do this, we had to manipulate our data slightly to make those values 0 and 1 instead of 'R' and 'L'

In [ ]:
data['p_throws'] = data['p_throws'].replace({'R': 1, 'L': 0})
data['stand'] = data['stand'].replace({'R': 1, 'L': 0})

### Grouping by at bat to find handedness
We then created an at-bat level grouping to find out how often each pitcher was facing someone of the same handedness as them. We do this by grouping by pitcher, game and at bat to ensure we separate out each at bat within a specific game.

In [ ]:
#group by at bat
at_bat_data = data.groupby(by = ['pitcher_ID', 'game_pk', 'at_bat_number']).agg(
                                    name = ('player_name', 'first'),
                                    p_throws = ('p_throws', 'first'),
                                    b_stands = ('stand', 'first')
)

In [ ]:
#get 0-1 value based on whether pitcher has a platoon advantage
at_bat_data['platoon_advantage'] = np.where(at_bat_data['p_throws'] == at_bat_data['b_stands'], 1, 0)

In [ ]:
#show resulting DF with platoon_advantage
at_bat_data = at_bat_data.reset_index()
at_bat_data

In [ ]:
# Group by pitcher id to get the total number of at bats each pitcher had platoon advantage
grouped = at_bat_data.groupby(by = 'pitcher_ID').sum()

#check that we have 3325 pitchers in index
print(len(grouped))

### Group by pitcher and season
Now, to facilitate our merge, we group by pitcher and season.

In [ ]:
#Group data from pitches into appearance
pitcher_level_data = data.groupby(by = ['pitcher_ID']).agg(
                            name = ('player_name', 'first'),
                            num_pitches = ('pitcher_ID', 'size'),
                            win_exp_delta = ('abs_delta_win_exp', np.mean))

### Merge total platoon in
Finally, we merge the total platoon data. It is ok for now that we have an integer as our platoon_advantage field, because we will divide that value by batters faced to get a platoon advantage percentage.

In [ ]:
# Merge data
pitcher_level_data_merged = pd.merge(pitcher_level_data, grouped, on = 'pitcher_ID', how = "inner").reset_index()
pitcher_level_data_merged.head()

# Save as csv

In [ ]:
pitcher_level_data_merged.to_csv('pitcher_level_data_bs.csv', index=False)

## Finding platoon advantage averages - lefties

---
We wanted to do one more computation in this notebook: finding the average percentage of time a pitcher of each handedness has the platoon advantage. Here, we do so for lefties.



In [ ]:
#get data for lefty pitchers
lefty_data = at_bat_data[at_bat_data['p_throws'] == 0]

In [ ]:
lefty_data.head()

In [ ]:
#get total percentage of at bats with lefty batter
lefty_average_platoon = len(lefty_data[lefty_data['b_stands'] == 0]) / len(lefty_data)
print('On average, lefty pitchers will face a lefty batter ' + str(round(lefty_average_platoon, 4)) + ' percent of the time.')

## Finding platoon advantage averages - righties
Here, we repeat the same process for right-handers

In [ ]:
#get data for lefty pitchers

righty_data = at_bat_data[at_bat_data['p_throws'] == 1]

In [ ]:
righty_average_platoon = len(righty_data[righty_data['b_stands'] == 0]) / len(righty_data)
print('On average, righty pitchers will face a righty batter ' + str(round(righty_average_platoon, 4)) + ' percent of the time.')